# Reverse2 - Volatility

### 1er étape :  Nom de la machine

Nous allons dans un premier temps executer une commande pour avoir les informations générales de la machine.

Pas beaucoup d'information sur notre nom de machine avec cette commande. On sait que le nom de la machine est contenue dans le registre et plus précisemment dans \REGISTRY\MACHINE\SYSTEM\

In [ ]:
import os
def windows_info():
    os.system("vol -f dump.vmem windows.info > window-info-output.txt")
    output = os.popen("vol -f dump.vmem windows.info").read()
    print(output)

windows_info()


vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName"
Cette commande nous permet d'afficher le contenu de la variable **ComputerName** contenue dans le registre MACHINE\SYSTEM
Nom de la machine : WIN-LO6FAF3DTFE

In [ ]:
def windows_getkey():
    os.system("vol -f dump.vmem windows.registry.printkey.PrintKey > window-getkey-output.txt")
    os.system('vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName" > window-getmachinename-output.txt')
    computername = os.popen('vol -f dump.vmem windows.registry.printkey. --offset 0xf8a000024010 --key "ControlSet001\Control\ComputerName\ComputerName"').read()
    print(computername)

windows_getkey()

### 2e étape :  Credentials

Pour ce faire on utilise hashdump qui nous récupère les hash des mots de passes de chaques sessions stocké.
Malheureusement les hash ne sont pas crackables via des outils en ligne tel que crackstation.

J'ai

In [ ]:
def getUserCredentials():
    os.system("vol -f dump.vmem windows.hashdump.Hashdump  > hashes.txt")
    output = os.popen("vol -f dump.vmem windows.hashdump.Hashdump").read()
    credentials = os.popen("vol -f dump.vmem windows.lsadump").read()
    print(credentials)

getUserCredentials()

### 3e étape :  Trouver l'ip de la machine sur le réseau
On va utiliser Netscan pour trouver facilement notre IP !

On tombe en boucle sur **192.168.202.131** : L'ip de la machine

In [ ]:
def getIp():
    os.system("vol -f dump.vmem windows.netscan.NetScan > netscan.txt")
    output = os.popen("vol -f dump.vmem windows.netscan.NetScan").read()
    print(output)

getIp()

### 4e étape : Savoir si le malware communique avec un serveur distant.
D'après le netscan, notre malware ne communique pas avec un serveur distant.

### 5e étape: trouver le malware dans la liste des process.
Pour ce faire, nous allons lancer un pstree. Cette commande nous permet d'afficher les processus en cours d'éxecution ainsi que leurs enfants.

In [ ]:
def getProcesses():
    os.system("vol -f dump.vmem windows.pstree > pstree.txt")
    os.system("vol -f dump.vmem windows.cmdline  > cmdline.txt")


getProcesses()

```
* 3820	2728	Rick And Morty	0xfa801b486b30	4	185	1	True	2018-08-04 19:32:55.000000 	N/A
** 3720	3820	vmware-tray.ex	0xfa801a4c5b30	8	147	1	True	2018-08-04 19:33:02.000000 	N/A
```
On repère vmware-tray.ex qui pourrait être notre malware et qui s'est installé lorsque notre user a lancé download.exe

On dump

In [45]:
def dump():
    os.system("vol -f dump.vmem windows.pslist --pid 3720 --dump")